In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.seasonal import seasonal_decompose

In [22]:
filename = "DummyHouses.pkl"
seed = 0

In [23]:
f = open(filename,"rb")
df = pickle.load(f)
X_feats = pickle.load(f)
Y_col = pickle.load(f)
f.close()

In [24]:
df.head()

,sold_days_before_last_sale,BEDRM_1,BEDRM_2,BEDRM_3,BEDRM_4,BEDRM_5,BEDRM_6,BEDRM_7,BEDRM_8,BEDRM_9,...,extra_rooms_10,extra_rooms_11,extra_rooms_12,extra_rooms_13,extra_rooms_14,extra_rooms_15,extra_rooms_16,extra_rooms_17,extra_rooms_20,MPlus
5,3058 days,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
16,2807 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
20,912 days,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
23,2151 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
24,2178 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


In [25]:
df[X_feats].head()

,sold_days_before_last_sale,BEDRM_1,BEDRM_2,BEDRM_3,BEDRM_4,BEDRM_5,BEDRM_6,BEDRM_7,BEDRM_8,BEDRM_9,...,extra_rooms_9,extra_rooms_10,extra_rooms_11,extra_rooms_12,extra_rooms_13,extra_rooms_14,extra_rooms_15,extra_rooms_16,extra_rooms_17,extra_rooms_20
5,3058 days,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,2807 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,912 days,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23,2151 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,2178 days,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
y = df[Y_col]

In [27]:
X = df[X_feats]
for col in X.columns:
    X[col] = (X[col] - min(X[col]))/(max(X[col]) - min(X[col]))

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [29]:
logreg = LogisticRegression(fit_intercept = False, C = 1e12, solver='lbfgs')
model_log = logreg.fit(X_train, y_train)
model_log

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
          fit_intercept=False, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [30]:
y_hat_prob = logreg.predict_proba(X_test)

In [31]:
y_hat_prob[:20]

array([[0.09938326, 0.90061674],
       [0.84188253, 0.15811747],
       [0.78834334, 0.21165666],
       [0.46714542, 0.53285458],
       [0.96964864, 0.03035136],
       [0.86193917, 0.13806083],
       [0.92897684, 0.07102316],
       [0.59777414, 0.40222586],
       [0.98147554, 0.01852446],
       [0.96787752, 0.03212248],
       [0.74600771, 0.25399229],
       [0.90772681, 0.09227319],
       [0.73594662, 0.26405338],
       [0.75570255, 0.24429745],
       [0.92223794, 0.07776206],
       [0.89401942, 0.10598058],
       [0.6479716 , 0.3520284 ],
       [0.83337217, 0.16662783],
       [0.60056016, 0.39943984],
       [0.80976537, 0.19023463]])

In [32]:
y_test.head(20)

40976      True
39124     False
53783     False
64726     False
94330     False
38662      True
26173      True
32461      True
85689     False
46759     False
84158     False
61371     False
8465       True
57149     False
53231     False
58209     False
103106    False
11332     False
97369     False
103931    False
Name: MPlus, dtype: bool

In [33]:
loglike = 0
for i, tf in enumerate(y_test):
    if tf:
        loglike += np.log(y_hat_prob[i][1])
    else:
        loglike += np.log(y_hat_prob[i][0])
print(loglike)

-2096.477380034954


In [34]:
k = len(X_feats)
print(k)

47


In [35]:
AIC = 2*k - 2*loglike
print(AIC)

4286.954760069908


In [36]:
lowest_AIC = 6247.723291471

In [37]:
lowest_AIC - AIC

1960.7685314010923